In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/HamoyeHQ/stage-f-03-restaurant-sales/master/data/train.csv')

In [3]:
def remove_outlier(col):
  sorted(col)
  Q1,Q3 = col.quantile([0.25,0.75])
  IQR = Q3-Q1
  lower_range = Q1 - (1.5 * IQR)
  upper_range = Q3 + (1.5 * IQR)
  return lower_range,upper_range

In [4]:
lowerrevenue, upperrevenue = remove_outlier(train_df['revenue'])
train_df['revenue'] = np.where(train_df['revenue']>upperrevenue ,upperrevenue,train_df['revenue'])
train_df['revenue'] = np.where(train_df['revenue']<lowerrevenue ,lowerrevenue,train_df['revenue'])
# df = train_df.copy()
# df= df.drop(columns='Id')
train_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [5]:
# subtract today's date from the open date
today = pd.to_datetime('today')
train_df[['days']] = train_df[['Open Date']].apply(pd.to_datetime)
train_df['days'] = (today - train_df['days'] ).dt.days
train_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P30,P31,P32,P33,P34,P35,P36,P37,revenue,days
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,5,3,4,5,5,4,3,4,5653753.0,7781
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,0,0,0,0,0,0,0,0,6923131.0,4647
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,0,0,0,0,0,0,0,0,2055379.0,2797
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,25,12,10,6,18,12,12,6,2675511.0,3198
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,5,1,3,2,3,4,3,3,4316715.0,4197


In [6]:
# Extract month index
def date_features(df):
  df['month'] = pd.DatetimeIndex(df['Open Date']).month
  return df


In [7]:
train_df=date_features(train_df)
train_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P31,P32,P33,P34,P35,P36,P37,revenue,days,month
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3,4,5,5,4,3,4,5653753.0,7781,7
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,0,0,0,0,0,0,0,6923131.0,4647,2
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,0,0,0,0,0,0,0,2055379.0,2797,3
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,12,10,6,18,12,12,6,2675511.0,3198,2
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,1,3,2,3,4,3,3,4316715.0,4197,5


In [8]:
final_df = train_df.loc[:,'City Group':'month']
final_df.head()

,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P31,P32,P33,P34,P35,P36,P37,revenue,days,month
0,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,...,3,4,5,5,4,3,4,5653753.0,7781,7
1,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,...,0,0,0,0,0,0,0,6923131.0,4647,2
2,Other,IL,2,4.0,2.0,5.0,2,3,5,5,...,0,0,0,0,0,0,0,2055379.0,2797,3
3,Other,IL,6,4.5,6.0,6.0,4,4,10,8,...,12,10,6,18,12,12,6,2675511.0,3198,2
4,Other,IL,3,4.0,3.0,4.0,2,2,5,5,...,1,3,2,3,4,3,3,4316715.0,4197,5


In [9]:
# Import label encoder 
from sklearn import preprocessing   
label_encoder = preprocessing.LabelEncoder() 
final_df['Type']= label_encoder.fit_transform(final_df['Type'])
final_df['City Group']= label_encoder.fit_transform(final_df['City Group'])
print(final_df['Type'].unique(),final_df['City Group'].unique()) 
final_df.head()

[2 1 0] [0 1]


,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P31,P32,P33,P34,P35,P36,P37,revenue,days,month
0,0,2,4,5.0,4.0,4.0,2,2,5,4,...,3,4,5,5,4,3,4,5653753.0,7781,7
1,0,1,4,5.0,4.0,4.0,1,2,5,5,...,0,0,0,0,0,0,0,6923131.0,4647,2
2,1,2,2,4.0,2.0,5.0,2,3,5,5,...,0,0,0,0,0,0,0,2055379.0,2797,3
3,1,2,6,4.5,6.0,6.0,4,4,10,8,...,12,10,6,18,12,12,6,2675511.0,3198,2
4,1,2,3,4.0,3.0,4.0,2,2,5,5,...,1,3,2,3,4,3,3,4316715.0,4197,5


In [10]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [11]:
x, y = final_df.drop('revenue', axis=1), final_df['revenue']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
x = pd.DataFrame(x)
print(x.shape,y.shape)

(137, 41) (137,)


In [12]:
params_xgb = {
    'learning_rate': [.1, .05, ],
    'colsample_bytree': [.3, .4, .5, .6],
    'max_depth': [1],
    'alpha': [3],
    'subsample': [.5],
    'n_estimators': [30, 70, 100, 200]
}

xgb_model = XGBRegressor()

cv = RepeatedKFold(n_splits=13, n_repeats=5, random_state=50)
xgb_regressor = GridSearchCV(xgb_model, params_xgb, scoring='neg_root_mean_squared_error', cv = cv, n_jobs = -1)
X_train = x
y_train = y
xgb_regressor.fit(X_train, y_train)
print(f'Optimal lr: {xgb_regressor.best_params_["learning_rate"]}')
print(f'Optimal colsample_bytree: {xgb_regressor.best_params_["colsample_bytree"]}')
print(f'Optimal n_estimators: {xgb_regressor.best_params_["n_estimators"]}')
print(f'max_depth: {xgb_regressor.best_params_["max_depth"]}')
print(f'Best score: {xgb_regressor.best_score_}')

Optimal lr: 0.05
Optimal colsample_bytree: 0.3
Optimal n_estimators: 100
max_depth: 1
Best score: -1686335.0564943708


In [13]:
model = XGBRegressor(learning_rate=xgb_regressor.best_params_["learning_rate"], 
                         colsample_bytree=xgb_regressor.best_params_["colsample_bytree"], 
                         max_depth=1, alpha=3, subsample=.5, 
                         n_estimators=xgb_regressor.best_params_["n_estimators"], n_jobs=-1)
model.fit(X_train, y_train)

XGBRegressor(alpha=3, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=1,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=3, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

#Testing

In [14]:
test_df = pd.read_csv('https://raw.githubusercontent.com/HamoyeHQ/stage-f-03-restaurant-sales/master/data/test.csv')
test_df.loc[test_df['Type']=='MB', 'Type'] = 'DT'
print(test_df['City Group'].unique(),test_df['City Group'].nunique())
print(test_df['Type'].unique(),test_df['Type'].nunique())
print(test_df['Type'].value_counts())
today = pd.to_datetime('today')
test_df[['days']] = test_df[['Open Date']].apply(pd.to_datetime) 
test_df['days'] = (today - test_df['days'] ).dt.days
test_df=date_features(test_df)
test_df.head()

['Other' 'Big Cities'] 2
['FC' 'IL' 'DT'] 3
FC    57019
IL    40447
DT     2534
Name: Type, dtype: int64


,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P30,P31,P32,P33,P34,P35,P36,P37,days,month
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,0,3574,1
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,0,3519,3
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,0,2562,10
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,0,4,0,0,0,0,0,0,2739,5
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,0,2653,7


In [15]:
test_df =test_df.loc[:,'City Group':'month']
# Import label encoder 
from sklearn import preprocessing   
label_encoder = preprocessing.LabelEncoder() 
test_df['Type']= label_encoder.fit_transform(test_df['Type'])
test_df['City Group']= label_encoder.fit_transform(test_df['City Group'])

x = scaler.transform(test_df)
x = pd.DataFrame(x)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.149797,-0.844929,-1.039607,-0.270816,-0.308701,-0.367578,-0.835796,-0.638471,-0.185001,-0.622801,...,-0.494874,-0.554863,-0.484658,-0.672851,-0.483667,-0.59269,-0.532553,-0.625925,-0.457925,-1.693405
1,1.149797,1.117025,-0.349892,-0.270816,-0.308701,-0.367578,-0.006056,-0.638471,-0.185001,-1.162824,...,-0.494874,-0.554863,-0.484658,-0.672851,-0.483667,-0.59269,-0.532553,-0.625925,-0.495446,-1.134377
2,-0.869718,-0.844929,-0.349892,-0.270816,-0.308701,-0.367578,-0.006056,-0.638471,-0.185001,-0.622801,...,-0.494874,-0.554863,-0.484658,-0.672851,-0.483667,-0.59269,-0.532553,-0.625925,-1.148297,0.822220
3,1.149797,1.117025,-0.694749,-0.270816,-0.308701,-0.367578,-0.006056,-0.168199,-0.185001,-0.622801,...,-0.494874,0.588238,-0.484658,-0.672851,-0.483667,-0.59269,-0.532553,-0.625925,-1.027550,-0.575350
4,1.149797,-0.844929,-0.694749,-0.270816,-0.308701,-0.367578,-0.835796,-0.638471,-0.185001,-0.622801,...,-0.494874,-0.554863,-0.484658,-0.672851,-0.483667,-0.59269,-0.532553,-0.625925,-1.086218,-0.016322


In [16]:
sub = pd.read_csv('https://raw.githubusercontent.com/HamoyeHQ/stage-f-03-restaurant-sales/master/data/sampleSubmission.csv')
submission = pd.DataFrame(columns=['Id','Prediction'])
submission['Id'] = sub['Id']
pred = model.predict(x)
submission['Prediction'] = pred
submission.head()

,Id,Prediction
0,0,4243208.50
1,1,3789861.50
2,2,3409068.75
3,3,3381859.25
4,4,3630832.00


In [17]:
submission.to_csv('../data/submission_xgb_000.csv',index=False)